In [1]:
import json
import subprocess
from pathlib import Path
import geopandas as gpd
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import thread_map
from pdal import Reader, Filter, Writer, Pipeline
from typing import Optional, Callable

In [2]:
def run_pdal(pipeline: Pipeline, pipe_name="pdal", args=[]):
    # don't use the PDAL pythonb bindings to run the pipeline, but make an external command call
    # this is because the PDAL python bindings are not thread safe
    # and they have the tendency to crash the python interpreter, while if they are a separate process there is no problem
    # for this use case we don't need to pass data between the python code and the PDAL pipeline so it pefectly fine to use the command line interface
    # it also enables to have interrupts that actually work, while the PDAL python bindings block the python interpreter
    with open(f"pipeline_{pipe_name}.json", "w") as f:
        f.write(pipeline.toJSON())
    cmd = ["pdal", "pipeline", f"pipeline_{pipe_name}.json"]
    cmd.extend(args)
    subprocess.run(cmd, check=True)

# Processing steps

In [3]:
base_dir = Path("/run/media/simone/Extreme SSD/spanish-lidar/lidar_cant")

In [4]:
raw_dir = base_dir / "0_raw_lidar"
denoise_dir = base_dir / "1_denoised_lidar"
ground_dir = base_dir / "2_ground_lidar"
dtm_dir = base_dir / "3_dtm_raster"
norm_dir = base_dir / "4_norm_lidar"
chm_dir = base_dir / "5_chm_raster"

# create directories
raw_dir.mkdir(exist_ok=True)
denoise_dir.mkdir(exist_ok=True)
ground_dir.mkdir(exist_ok=True)
dtm_dir.mkdir(exist_ok=True)
norm_dir.mkdir(exist_ok=True)
chm_dir.mkdir(exist_ok=True)

## 0) Create raw tindex

In [5]:
file_spec = str(raw_dir / "*.laz")
raw_tindex_path = raw_dir / "raw_tindex.gpkg"

In [6]:
!pdal tindex create -f GPKG "{str(raw_tindex_path)}" --filespec "{file_spec}"

Warning 1: A geometry of type MULTIPOLYGON is inserted into layer PNOA_2023_CANT_376-4775_NPC02 of geometry type POLYGON, which is not normally allowed by the GeoPackage specification, but the driver will however do it. To create a conformant GeoPackage, if using ogr2ogr, the -nlt option can be used to override the layer geometry type. This warning will no longer be emitted for this combination of layer and feature geometry type.


In [6]:
raw_tindex = gpd.read_file(raw_tindex_path)

In [7]:
cloud_raw = Path(raw_tindex.iloc[0].location)

## 1) Denoise 

In [8]:
cloud_denoise = denoise_dir / f"denoise__{cloud_raw.stem}.copc.laz"

In [40]:
denoise_pipe = Pipeline(
    [
        Reader.las(cloud_raw),
        # Filter.outlier(
        #     method="radius", min_k=12, radius=2.0
        # ),  
        Filter.expression(expression="Classification!=7"),
        Writer.copc(cloud_denoise),
    ]
)

In [36]:
run_pdal(denoise_pipe, "denoise")

## 2) Ground

In [10]:
cloud_ground = ground_dir / f"ground__{cloud_raw.stem}.copc.laz"

In [11]:
ground_pipe = Pipeline(
    [
        Reader.copc(cloud_denoise),
        Filter.expression(expression="Classification==2"),
        Writer.copc(cloud_ground),
    ]
)

In [40]:
run_pdal(ground_pipe, "ground")

## 3) DTM

In [12]:
cloud_dtm = dtm_dir / f"dtm__{cloud_raw.stem}.tif"

In [ ]:
dtm_pipe = Pipeline(
    [
        Reader.copc(cloud_ground),
        Filter.delaunay(),
        Filter.faceraster(resolution=1),
        Writer.raster(
            cloud_dtm#, gdalopts="COMPRESS=ZSTD", nodata="NaN", data_type="float32"
        ),
    ]
)

In [43]:
run_pdal(dtm_pipe, "dtm")

## 4) Normalization

In [29]:
cloud_norm = norm_dir / f"norm__{cloud_raw.stem}.copc.laz"
hag_dir = norm_dir / "hag"
hag_dir.mkdir(exist_ok=True)
cloud_hag = hag_dir / f"hag__{cloud_raw.stem}.copc.laz"

In [45]:
norm_pipe = Pipeline(
    [
        Reader.copc(cloud_ground),
        # Filter.hag_dem(raster=cloud_dtm),
        Filter.hag_delaunay(count=30),
        Writer.copc(cloud_hag, tag="hag", extra_dims="HeightAboveGround=float32"),
        Filter.ferry(dimensions="HeightAboveGround=>Z"),
        Filter.expression(
            expression="Z >= 0 && Z <= 30"
        ),  # remove normalization artifacts
        Writer.copc(cloud_norm),
    ]
)

In [31]:
run_pdal(norm_pipe, "norm")

## 5) CHM

In [16]:
cloud_chm = chm_dir / f"chm__{cloud_raw.stem}.tif"

In [ ]:
chm_pipe = Pipeline(
    [
        Reader.copc(cloud_norm),
        Writer.gdal(
            filename=cloud_chm,
            resolution=1,
            output_type="max",
            # gdalopts="COMPRESS=ZSTD",
            # nodata="NaN",
            # data_type="float32",
        ),
    ]
)

In [49]:
run_pdal(chm_pipe, "chm")

# Run everything

In [55]:
def process_tindex_parallel(
    tindex: gpd.GeoDataFrame,
    pipeline: Pipeline,
    pipe_name: str,
    out_dir: Path,
    output_extension: str = ".copc.laz",
    input_command="--readers.copc.filename",
    output_command="--writers.copc.filename",
    extra_args: Optional[Callable[[str], str]] = None,
    max_workers: int = 8
) -> gpd.GeoDataFrame:
    def process_row(row):
        input_file = Path(row.location)
        base_name = row.base_name
        output_file = out_dir / f"{pipe_name}__{base_name}{output_extension}"
        args = extra_args(base_name) if extra_args else []
        if not output_file.exists():
            try:
                run_pdal(
                    pipeline,
                    pipe_name,
                    [input_command, str(input_file), output_command, str(output_file), *args],
                )
            except subprocess.CalledProcessError as e:
                print(f"Error processing {input_file}: {e}")
        return output_file

    output = thread_map(
        process_row,
        tindex.itertuples(),
        desc=f"Processing {pipe_name}",
        total=len(tindex),
        max_workers=max_workers,
    )
    new_tindex = tindex.copy()
    new_tindex["location"] = output
    return new_tindex

In [56]:
def processing_pipeline(tindex, max_workers=8):
    denoise_tindex = process_tindex_parallel(
        tindex,
        denoise_pipe,
        "denoise",
        denoise_dir,
        input_command="--readers.las.filename",
        max_workers=max_workers
    )
    denoise_tindex.to_file(denoise_dir / "denoise_tindex.gpkg")
    # denoise_tindex = gpd.read_file(denoise_dir / "denoise_tindex.gpkg")

    ground_tindex = process_tindex_parallel(
        denoise_tindex,
        ground_pipe,
        "ground",
        ground_dir,
        max_workers=max_workers
    )
    ground_tindex.to_file(ground_dir / "ground_tindex.gpkg")

    dtm_tindex = process_tindex_parallel(
        ground_tindex,
        dtm_pipe,
        "dtm",
        dtm_dir,
        output_extension=".tif",
        output_command="--writers.raster.filename",
        max_workers=max_workers
    )
    dtm_tindex.to_file(ground_dir / "dtm_tindex.gpkg")

    norm_tindex = process_tindex_parallel(
        denoise_tindex,
        norm_pipe,
        "norm",
        norm_dir,
        extra_args = lambda base_name: [
            # f"--filters.hag_dem.raster={dtm_dir / f'dtm__{base_name}.tif'}]",
              "--stage.hag.filename", str(hag_dir / f"hag__{base_name}.copc.laz")],
        max_workers=max_workers
    )
    norm_tindex.to_file(norm_dir / "norm_tindex.gpkg")

    chm_tindex = process_tindex_parallel(
        norm_tindex,
        chm_pipe,
        "chm",
        chm_dir,
        output_extension=".tif",
        output_command="--writers.gdal.filename",
        max_workers=max_workers
    )
    chm_tindex.to_file(chm_dir / "chm_tindex.gpkg")
    

In [48]:
raw_tindex['base_name'] = raw_tindex.location.apply(lambda x: Path(x).stem)

In [49]:
processing_pipeline(raw_tindex.iloc[:2])

Processing denoise:   0%|          | 0/2 [00:00<?, ?it/s]

Processing ground:   0%|          | 0/2 [00:00<?, ?it/s]

Processing dtm:   0%|          | 0/2 [00:00<?, ?it/s]

Processing norm:   0%|          | 0/2 [00:00<?, ?it/s]

Processing chm:   0%|          | 0/2 [00:00<?, ?it/s]

In [57]:
processing_pipeline(raw_tindex, max_workers=12)

Processing denoise:   0%|          | 0/98 [00:00<?, ?it/s]

Processing ground:   0%|          | 0/98 [00:00<?, ?it/s]

Processing dtm:   0%|          | 0/98 [00:00<?, ?it/s]

Processing norm:   0%|          | 0/98 [00:00<?, ?it/s]

PDAL: All points collinear



...

Error processing /run/media/simone/Extreme SSD/spanish-lidar/lidar_cant/4_norm_lidar/norm__PNOA_2023_CANT_372-4777_NPC02.copc.laz: Command '['pdal', 'pipeline', 'pipeline_chm.json', '--readers.copc.filename', '/run/media/simone/Extreme SSD/spanish-lidar/lidar_cant/4_norm_lidar/norm__PNOA_2023_CANT_372-4777_NPC02.copc.laz', '--writers.gdal.filename', '/run/media/simone/Extreme SSD/spanish-lidar/lidar_cant/5_chm_raster/chm__PNOA_2023_CANT_372-4777_NPC02.tif']' returned non-zero exit status 1.


PDAL: Unable to open '/run/media/simone/Extreme SSD/spanish-lidar/lidar_cant/4_norm_lidar/norm__PNOA_2023_CANT_374-4776_NPC02.copc.laz'. File does not exist.



Error processing /run/media/simone/Extreme SSD/spanish-lidar/lidar_cant/4_norm_lidar/norm__PNOA_2023_CANT_374-4776_NPC02.copc.laz: Command '['pdal', 'pipeline', 'pipeline_chm.json', '--readers.copc.filename', '/run/media/simone/Extreme SSD/spanish-lidar/lidar_cant/4_norm_lidar/norm__PNOA_2023_CANT_374-4776_NPC02.copc.laz', '--writers.gdal.filename', '/run/media/simone/Extreme SSD/spanish-lidar/lidar_cant/5_chm_raster/chm__PNOA_2023_CANT_374-4776_NPC02.tif']' returned non-zero exit status 1.


PDAL: Unable to open '/run/media/simone/Extreme SSD/spanish-lidar/lidar_cant/4_norm_lidar/norm__PNOA_2023_CANT_375-4777_NPC02.copc.laz'. File does not exist.



Error processing /run/media/simone/Extreme SSD/spanish-lidar/lidar_cant/4_norm_lidar/norm__PNOA_2023_CANT_375-4777_NPC02.copc.laz: Command '['pdal', 'pipeline', 'pipeline_chm.json', '--readers.copc.filename', '/run/media/simone/Extreme SSD/spanish-lidar/lidar_cant/4_norm_lidar/norm__PNOA_2023_CANT_375-4777_NPC02.copc.laz', '--writers.gdal.filename', '/run/media/simone/Extreme SSD/spanish-lidar/lidar_cant/5_chm_raster/chm__PNOA_2023_CANT_375-4777_NPC02.tif']' returned non-zero exit status 1.
